In [3]:
import pandas as pd
import pymysql

# === 설정 ===
CSV_PATH = r"./Supabase Snippet Hotspot District Codes.csv"
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_USER = "root"
MYSQL_PASSWORD = "root"
MYSQL_DB = "project_db"

# accident_hotspots에서 CSV accident_id와 매칭되는 컬럼명
HOTSPOTS_KEY_COL = "accident_id"   # 예: "id"면 여기만 바꿔

# === CSV 로드 ===
df = pd.read_csv(CSV_PATH)

# 컬럼 존재 체크
need_cols = {"accident_id", "district_code"}
missing = need_cols - set(df.columns)
if missing:
    raise ValueError(f"CSV에 컬럼이 없음: {missing}. 현재 컬럼: {list(df.columns)}")

# 값 정리: 공백/NaN 제거, 문자열 정규화
df["district_code"] = df["district_code"].astype(str).str.strip()
df = df[df["district_code"].notna() & (df["district_code"] != "")]

# accident_id도 정리
df["accident_id"] = pd.to_numeric(df["accident_id"], errors="coerce")
df = df[df["accident_id"].notna()]
df["accident_id"] = df["accident_id"].astype(int)

# 중복 키가 있으면 마지막 값 기준으로 정리
df = df.drop_duplicates(subset=["accident_id"], keep="last")

print("CSV rows to apply:", len(df))

# === MySQL 연결 ===
conn = pymysql.connect(
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DB,
    autocommit=False,
    charset="utf8mb4",
)
try:
    with conn.cursor() as cur:
        # 0) 업데이트 전 UNKNOWN 개수
        cur.execute("SELECT COUNT(*) FROM accident_hotspots WHERE district_code='UNKNOWN'")
        before_unknown = cur.fetchone()[0]
        print("UNKNOWN before:", before_unknown)

        # 1) 임시 테이블 생성 (세션 동안만 존재)
        cur.execute("""
            CREATE TEMPORARY TABLE tmp_hotspot_district (
                accident_id BIGINT NOT NULL,
                district_code VARCHAR(20) NOT NULL,
                PRIMARY KEY (accident_id)
            ) ENGINE=InnoDB
        """)

        # 2) 배치 insert (빠르게)
        rows = list(df[["accident_id", "district_code"]].itertuples(index=False, name=None))
        insert_sql = """
            INSERT INTO tmp_hotspot_district (accident_id, district_code)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE district_code = VALUES(district_code)
        """
        cur.executemany(insert_sql, rows)
        print("Inserted into temp:", cur.rowcount)

        # 3) UNKNOWN만 업데이트
        update_sql = f"""
            UPDATE accident_hotspots h
            JOIN tmp_hotspot_district t
              ON h.{HOTSPOTS_KEY_COL} = t.accident_id
            SET h.district_code = t.district_code
            WHERE h.district_code = 'UNKNOWN'
        """
        cur.execute(update_sql)
        updated = cur.rowcount
        print("Updated rows:", updated)

        # 4) 업데이트 후 UNKNOWN 개수
        cur.execute("SELECT COUNT(*) FROM accident_hotspots WHERE district_code='UNKNOWN'")
        after_unknown = cur.fetchone()[0]
        print("UNKNOWN after:", after_unknown)

    conn.commit()
    print("DONE: committed.")
except Exception:
    conn.rollback()
    raise
finally:
    conn.close()

CSV rows to apply: 1984
UNKNOWN before: 1884
Inserted into temp: 1984
Updated rows: 1884
UNKNOWN after: 0
DONE: committed.
